In [2]:
!pip3 install pandas
!pip3 install pendulum
!pip3 install keras_tuner

     |████████████████████████████████| 11.7 MB 10.3 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 155 kB 28.4 MB/s eta 0:00:01
     |████████████████████████████████| 489 kB 39.4 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 98 kB 8.3 MB/s  eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [3]:
import tensorflow as tf

In [4]:


BASE_MODEL_INPUT_SHAPE = (600,600,3)

cifar = tf.keras.datasets.cifar10.load_data()
(x_train, y_train), (x_test, y_test) = cifar


mod_with_fc_raw = tf.keras.applications.efficientnet.EfficientNetB7(
    include_top=True, weights='imagenet', input_tensor=None,
    input_shape = BASE_MODEL_INPUT_SHAPE, pooling='max', classes=1000
)

# Make the deepest conv2d layer trainable, leave everything else
# as not trainable
for layer in mod_with_fc_raw.layers:
    layer.trainable = False
# Last conv2d layer. This we want to train .
mod_with_fc_raw.layers[-6].trainable = True

# Create the final base model
# (remove the final Dense and BatchNormalization layers ...) 
efficient_net_b_7_transferable_base_model =\
    tf.keras.Model(inputs=mod_with_fc_raw.layers[0].input, 
                    outputs=mod_with_fc_raw.layers[-3].output)


170508288/170498071 [==============================] - 12s 0us/step


2022-02-18 05:02:16.371637: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-18 05:02:16.516974: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-18 05:02:16.517673: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-18 05:02:16.520417: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-18 05:02:16.521077: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read f

268337152/268326632 [==============================] - 11s 0us/step


In [5]:
TRAINING_SET_SIZE = 1000

y_train_ohe = tf.one_hot([i[0] for i in  y_train],10)
indexes_for_rows = tf.range(0,y_train.shape[0])
shuffled_indexes = tf.random.shuffle(indexes_for_rows)
selected_indexes = shuffled_indexes[:TRAINING_SET_SIZE]
selected_x_train = x_train[selected_indexes,:,:,:]
selected_y_train_ohe = y_train_ohe.numpy()[selected_indexes,:]

In [6]:
from residualmlp.residual_mlp import ResidualMLP

In [7]:
from inspect import getfullargspec
getfullargspec(ResidualMLP)

FullArgSpec(args=['self', 'problem_type', 'learning_rate', 'minimum_learning_rate', 'maximum_learning_rate', 'number_of_learning_rates_to_try', 'input_shape', 'bw_images', 'base_model', 'base_model_input_shape', 'base_model_hyperparameters', 'flatten_after_base_model', 'blocks', 'minimum_number_of_blocks', 'maximum_number_of_blocks', 'minimum_number_of_layers_per_block', 'maximum_number_of_layers_per_block', 'minimum_neurons_per_block_layer', 'maximum_neurons_per_block_layer', 'n_options_of_neurons_per_layer_to_try', 'minimum_neurons_per_block_layer_decay', 'maximum_neurons_per_block_layer_decay', 'residual_bypass_dense_layers', 'b_norm_or_dropout_residual_bypass_layers', 'dropout_rate_for_bypass_layers', 'minimum_dropout_rate_for_bypass_layers', 'maximim_dropout_rate_for_bypass_layers', 'n_options_dropout_rate_for_bypass_layers', 'inter_block_layers_per_block', 'minimum_inter_block_layers_per_block', 'maximum_inter_block_layers_per_block', 'n_options_inter_block_layers_per_block', 'b_

In [16]:
# [batch, timesteps, feature]

BATCH = 5
STEPS = 20
NEURONS_GRU_MLP = 50
NEURONS_EFF_MLP = 50

inp = tf.keras.layers.Input(shape=(32,32,3))
x = tf.keras.layers.Resizing(600,600)(inp)
y = inp

x = efficient_net_b_7_transferable_base_model(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(NEURONS_EFF_MLP,
                        activation=tf.keras.activations.relu, 
                        kernel_initializer=tf.keras.initializers.GlorotNormal)(x)
x_residual = x

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(NEURONS_EFF_MLP,
                        activation=tf.keras.activations.relu, 
                        kernel_initializer=tf.keras.initializers.GlorotNormal)(x)
x = tf.keras.layers.Concatenate(axis=1)([x,x_residual])

y_ch0 = y[:,:,:,0] # tf.keras.layers.Flatten()
y_ch1 = y[:,:,:,1] # tf.keras.layers.Flatten()(
y_ch2 = y[:,:,:,2] # tf.keras.layers.Flatten()()

y_ch0, y_ch0_state  = tf.keras.layers.GRU(32,input_shape=(None,32,32),return_sequences=True, return_state=True)(y_ch0)
y_ch1, y_ch1_state  = tf.keras.layers.GRU(32,input_shape=(None,32,32),return_sequences=True, return_state=True)(y_ch1)
y_ch2, y_ch2_state  = tf.keras.layers.GRU(32,input_shape=(None,32,32),return_sequences=True, return_state=True)(y_ch2)

y_ch0 = tf.keras.layers.GRU(32,input_shape=(None,32,32))([y_ch0,y_ch0_state])
y_ch1 = tf.keras.layers.GRU(32,input_shape=(None,32,32))([y_ch1,y_ch1_state])
y_ch2 = tf.keras.layers.GRU(32,input_shape=(None,32,32))([y_ch2,y_ch2_state])

y = tf.keras.layers.Concatenate(axis=1)([y_ch0,y_ch1,y_ch2])

y = tf.keras.layers.BatchNormalization()(y)
y = tf.keras.layers.Dense(NEURONS_GRU_MLP,
                        activation=tf.keras.activations.relu, 
                        kernel_initializer=tf.keras.initializers.GlorotNormal)(y)

y_residual = y

y = tf.keras.layers.BatchNormalization()(y)
y = tf.keras.layers.Dense(NEURONS_GRU_MLP,
                        activation=tf.keras.activations.relu, 
                        kernel_initializer=tf.keras.initializers.GlorotNormal)(y)
y = tf.keras.layers.Concatenate(axis=1)([y,y_residual])

final_base_model_output = tf.keras.layers.Concatenate(axis=1)([x,y])



concatenated_efficientnet_gru_model = tf.keras.Model(inputs=inp,outputs=final_base_model_output)

concatenated_efficientnet_gru_model(x_train[:5])


<tf.Tensor: shape=(5, 200), dtype=float32, numpy=
array([[0.        , 0.20556308, 0.10319729, 0.        , 0.        ,
        0.        , 0.        , 0.23095171, 0.        , 0.20087379,
        0.22799511, 0.        , 0.        , 0.        , 0.11601258,
        0.        , 0.        , 0.12606584, 0.15618055, 0.        ,
        0.08006047, 0.        , 0.        , 0.        , 0.04185324,
        0.        , 0.06248565, 0.24495897, 0.1633819 , 0.18569756,
        0.09699996, 0.16946535, 0.28391987, 0.        , 0.05864558,
        0.10620824, 0.        , 0.0805537 , 0.        , 0.24124733,
        0.        , 0.        , 0.        , 0.        , 0.03514363,
        0.        , 0.        , 0.        , 0.        , 0.18335459,
        0.28679764, 0.26123786, 0.        , 0.1726978 , 0.38940158,
        0.08559008, 0.15339065, 0.0581256 , 0.09198355, 0.32420203,
        0.        , 0.15500632, 0.        , 0.41381875, 0.13975704,
        0.26075855, 0.        , 0.18362373, 0.        , 0.5047065 

In [20]:
concatenated_efficientnet_gru_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_6 (Sli (None, 32, 32)       0           input_4[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_7 (Sli (None, 32, 32)       0           input_4[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_8 (Sli (None, 32, 32)       0           input_4[0][0]                    
____________________________________________________________________________________________

In [18]:
Date = '2022-02-18_00_23'
PROBLEM_TYPE = 'classification'
NUMBER_OF_CLASSES = 10
INPUT_SHAPE = (32, 32, 3)
BASE_MODEL_INPUT_SHAPE = (32, 32, 3)
PROJECT_NAME = 'CIFAR10_GRU-EfficientNetB7-ResidualMLP_NAS_THIRD_PASS_SEARCH'
TRAINING_SET_SIZE = 1000
PATIENCE = 25
PATIENCE_MIN_DELTA = 1e-05
BATCH_SIZE = 50
MAX_EPOCHS = 150
RESULTS_DIR_FOR_SEARCH = '2022-02-12_14_48_CIFAR10_EfficientNetB7-ResidualMLP_NAS_THIRD_PASS_SEARCH_SEARCH_RUN'
MINIMUM_LEARNING_RATE = 0.0001
MAXIMUM_LEARNING_RATE = 0.02
NUMBER_OF_LEARNING_RATES_TO_TRY = 7
MINIMUM_NUMBER_OF_BLOCKS = 2
MAXIMUM_NUMBER_OF_BLOCKS = 3
MINIMUM_NUMBER_OF_LAYERS_PER_BLOCK = 2
MAXIMUM_NUMBER_OF_LAYERS_PER_BLOCK = 4
MINIMUM_NEURONS_PER_BLOCK_LAYER = 70
MAXIMUM_NEURONS_PER_BLOCK_LAYER = 140
N_OPTIONS_OF_NEURONS_PER_LAYER_TO_TRY = 7
MINIMUM_NEURONS_PER_BLOCK_LAYER_DECAY = 15
MAXIMUM_NEURONS_PER_BLOCK_LAYER_DECAY = 35
MINIMUM_DROPOUT_RATE_FOR_BYPASS_LAYERS = 0.2
MAXIMIM_DROPOUT_RATE_FOR_BYPASS_LAYERS = 0.6
N_OPTIONS_DROPOUT_RATE_FOR_BYPASS_LAYERS = 3
MINIMUM_INTER_BLOCK_LAYERS_PER_BLOCK = 0
MAXIMUM_INTER_BLOCK_LAYERS_PER_BLOCK = 85
N_OPTIONS_INTER_BLOCK_LAYERS_PER_BLOCK = 7
MINIMUM_DROPOUT_RATE = 0.2
MAXIMUM_DROPOUT_RATE = 0.6
N_OPTIONS_DROPOUT_RATE = 5
MINIMUM_FINAL_DENSE_LAYERS = 33
MAXIMUM_FINAL_DENSE_LAYERS = 300
N_OPTIONS_FINAL_DENSE_LAYERS = 7

In [21]:
model_builder = ResidualMLP(
                    problem_type = PROBLEM_TYPE,
                    minimum_learning_rate = MINIMUM_LEARNING_RATE, 
                    maximum_learning_rate = MAXIMUM_LEARNING_RATE, 
                    number_of_learning_rates_to_try =
                        NUMBER_OF_LEARNING_RATES_TO_TRY, 
                    input_shape = INPUT_SHAPE, 
                    bw_images = False, 
                    base_model =\
                        concatenated_efficientnet_gru_model, 
                    base_model_input_shape = BASE_MODEL_INPUT_SHAPE, 
                    flatten_after_base_model = False, 
                    minimum_number_of_blocks = MINIMUM_NUMBER_OF_BLOCKS, 
                    maximum_number_of_blocks = MAXIMUM_NUMBER_OF_BLOCKS, 
                    minimum_number_of_layers_per_block =
                        MINIMUM_NUMBER_OF_LAYERS_PER_BLOCK, 
                    maximum_number_of_layers_per_block =
                        MAXIMUM_NUMBER_OF_LAYERS_PER_BLOCK,
                    minimum_neurons_per_block_layer =
                        MINIMUM_NEURONS_PER_BLOCK_LAYER, 
                    maximum_neurons_per_block_layer =
                        MAXIMUM_NEURONS_PER_BLOCK_LAYER, 
                    n_options_of_neurons_per_layer_to_try =
                        N_OPTIONS_OF_NEURONS_PER_LAYER_TO_TRY, 
                    minimum_neurons_per_block_layer_decay =
                        MINIMUM_NEURONS_PER_BLOCK_LAYER_DECAY, 
                    maximum_neurons_per_block_layer_decay = 
                        MAXIMUM_NEURONS_PER_BLOCK_LAYER_DECAY, 
                    minimum_dropout_rate_for_bypass_layers =
                        MINIMUM_DROPOUT_RATE_FOR_BYPASS_LAYERS, 
                    maximim_dropout_rate_for_bypass_layers =
                        MAXIMIM_DROPOUT_RATE_FOR_BYPASS_LAYERS, 
                    n_options_dropout_rate_for_bypass_layers =
                        N_OPTIONS_DROPOUT_RATE_FOR_BYPASS_LAYERS,
                    minimum_inter_block_layers_per_block =
                        MINIMUM_INTER_BLOCK_LAYERS_PER_BLOCK, 
                    maximum_inter_block_layers_per_block =
                        MAXIMUM_INTER_BLOCK_LAYERS_PER_BLOCK,
                    n_options_inter_block_layers_per_block =  \
                        N_OPTIONS_INTER_BLOCK_LAYERS_PER_BLOCK,
                    minimum_dropout_rate = MINIMUM_DROPOUT_RATE, 
                    maximum_dropout_rate = MAXIMUM_DROPOUT_RATE,
                    n_options_dropout_rate = N_OPTIONS_DROPOUT_RATE, 
                    minimum_final_dense_layers =
                        MINIMUM_FINAL_DENSE_LAYERS,
                    maximum_final_dense_layers =
                        MAXIMUM_FINAL_DENSE_LAYERS, 
                    n_options_final_dense_layers =
                        N_OPTIONS_FINAL_DENSE_LAYERS, 
                    number_of_classes = NUMBER_OF_CLASSES,
                    final_activation = tf.keras.activations.softmax)

In [25]:
import keras_tuner as kt

In [26]:
tuner = kt.Hyperband(
    model_builder.build_auto_residual_mlp,
    objective='val_loss',
    project_name = PROJECT_NAME,
    max_epochs = MAX_EPOCHS,
    hyperband_iterations = 2)

All permutations:
     number_of_blocks  layers_per_block  neurons_per_block_layer  \
0                   2                 2                       70   
1                   2                 2                       70   
2                   2                 2                       70   
3                   2                 2                       70   
4                   2                 2                       70   
..                ...               ...                      ...   
877                 3                 4                      140   
878                 3                 4                      140   
879                 3                 4                      140   
880                 3                 4                      140   
881                 3                 4                      140   

     neurons_per_block_layer_decay  
0                               15  
1                               16  
2                               17  
3                

In [ ]:
tuner.search(x=selected_x_train,  
             y=selected_y_train_ohe,
             epochs=MAX_EPOCHS,
             batch_size=BATCH_SIZE, 
             callbacks=[
                    tf.keras.callbacks.EarlyStopping(
                        monitor="val_loss",
                        patience=PATIENCE,
                        min_delta=PATIENCE_MIN_DELTA,
                        restore_best_weights=True,
                    ),
                    tensorboard_callback_search,
                ],
             validation_split=0.3)

In [ ]:
# Train base model:

In [ ]:
import os
BATCH_SIZE = 5
PATIENCE = 5
PATIENCE_MIN_DELTA = .002
RESULTS_DIR_FOR_FINAL_MODEL = "2022-02-17_efficientnet_concat_gru_model"

logdir_final_model = os.path.join("logs",
                                  RESULTS_DIR_FOR_FINAL_MODEL + "_TB")
tensorboard_callback_search =\
    tf.keras.callbacks.TensorBoard(logdir_final_model, histogram_freq=1)

tuner.search(x=selected_x_train,  
             y=selected_y_train_ohe,
             epochs=MAX_EPOCHS,
             batch_size=BATCH_SIZE, 
             callbacks=[
                    tf.keras.callbacks.EarlyStopping(
                        monitor="val_loss",
                        patience=PATIENCE,
                        min_delta=PATIENCE_MIN_DELTA,
                        restore_best_weights=True,
                    ),
                    tensorboard_callback_search,
                ],
             validation_split=0.3)


2022-02-18 05:34:59.292340: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-02-18 05:34:59.292392: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-02-18 05:34:59.635411: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-02-18 05:34:59.635651: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1749] CUPTI activity buffer flushed



Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
learning_rate     |0.0017671         |?                 
blocks            |62                |?                 
bypass_layers_u...|70                |?                 
inter_block_layers|85                |?                 
final_dense_layers|77                |?                 
b_norm_or_dropo...|bnorm             |?                 
dropout_rate_fo...|0.2               |?                 
b_norm_or_dropo...|bnorm             |?                 
dropout_rate      |0.2               |?                 
tuner/epochs      |2                 |?                 
tuner/initial_e...|0                 |?                 
tuner/bracket     |4                 |?                 
tuner/round       |0                 |?                 

All permutations:
     number_of_blocks  layers_per_block  neurons_per_block_layer  \
0                   2                 2                       70   
1                   

/usr/local/lib/python3.8/dist-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
2022-02-18 05:35:02.525136: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/2
  1/140 [..............................] - ETA: 1:03:19 - loss: 3.4376 - top_1_categorical_accuracy: 0.0000e+00 - top_2_categorical_accuracy: 0.0000e+00 - top_3_categorical_accuracy: 0.0000e+00 - top_4_categorical_accuracy: 0.0000e+00 - top_5_categorical_accuracy: 0.2000 - top_6_categorical_accuracy: 0.4000 - top_7_categorical_accuracy: 0.6000 - top_8_categorical_accuracy: 0.8000 - top_9_categorical_accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - accuracy: 0.0000e+00

2022-02-18 05:35:31.244088: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-02-18 05:35:31.244150: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


  2/140 [..............................] - ETA: 16:15 - loss: 3.1125 - top_1_categorical_accuracy: 0.0000e+00 - top_2_categorical_accuracy: 0.1000 - top_3_categorical_accuracy: 0.1000 - top_4_categorical_accuracy: 0.2000 - top_5_categorical_accuracy: 0.4000 - top_6_categorical_accuracy: 0.6000 - top_7_categorical_accuracy: 0.8000 - top_8_categorical_accuracy: 0.9000 - top_9_categorical_accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - accuracy: 0.0000e+00              

2022-02-18 05:35:36.978760: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-02-18 05:35:36.979346: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1749] CUPTI activity buffer flushed
2022-02-18 05:35:37.074848: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 5643 callback api events and 5414 activity events. 
2022-02-18 05:35:37.193003: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-02-18 05:35:37.326467: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/2022-02-17_efficientnet_concat_gru_model_TB/4b7f368d3655dc6743263b3ddadd936f/execution0/train/plugins/profile/2022_02_18_05_35_37

2022-02-18 05:35:37.405368: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/2022-02-17_efficientnet_concat_gru_model_TB/4b7f368d3655dc6743263b3ddadd936f/execution0/train/plugins

140/140 [==============================] - 839s 6s/step - loss: 2.4020 - top_1_categorical_accuracy: 0.2271 - top_2_categorical_accuracy: 0.3914 - top_3_categorical_accuracy: 0.4986 - top_4_categorical_accuracy: 0.6071 - top_5_categorical_accuracy: 0.6857 - top_6_categorical_accuracy: 0.7571 - top_7_categorical_accuracy: 0.8500 - top_8_categorical_accuracy: 0.9143 - top_9_categorical_accuracy: 0.9714 - precision: 0.2908 - recall: 0.0814 - accuracy: 0.0000e+00 - val_loss: 2.4666 - val_top_1_categorical_accuracy: 0.2200 - val_top_2_categorical_accuracy: 0.3733 - val_top_3_categorical_accuracy: 0.4933 - val_top_4_categorical_accuracy: 0.6167 - val_top_5_categorical_accuracy: 0.6767 - val_top_6_categorical_accuracy: 0.7533 - val_top_7_categorical_accuracy: 0.8233 - val_top_8_categorical_accuracy: 0.9267 - val_top_9_categorical_accuracy: 0.9667 - val_precision: 0.1600 - val_recall: 0.0267 - val_accuracy: 0.0000e+00
Epoch 2/2
140/140 [==============================] - ETA: 0s - loss: 1.9645 

In [ ]:
# Add stanza for getting best model

In [ ]:
!pip3 install matplotlib
import pandas as pd
import matplotlib.pyplot as plt
hy = pd.DataFrame(history.history)

In [ ]:
hy

In [ ]:
hy[["top_k_categorical_accuracy","val_top_k_categorical_accuracy"]].plot()

In [ ]:
hy[["loss","val_loss"]].plot()